# Analysis layers

In [1]:
%run '00_Imports-and-settings.ipynb'

Numpy Version:		 1.19.0
Pandas Version:		 1.0.5
Matplotlib Version:	 3.2.2


In [2]:
# helper function to get model name
def get_model_name(model_string):
    if 'ResNet' in model_string:
        model_name = 'ResNet20_CIFAR-10'
    elif 'LeNet' in model_string:
        model_name = 'LeNet-MNIST'
    else:
        model_name = 'unknown'
        print('unknown model name')
    return model_name

## Plot layers for different optimizations - same model, same MCU

In [3]:
df = pd.read_pickle('results_layers/A_aggregated_benchmarking-layers_results.pkl')

In [4]:
df['model_type'] = df['model_type'].str.replace('_','-')
df['MCU'] = df['MCU'].str.replace('_','-')

In [5]:
df.columns

Index(['time', 'MCU', 'model', 'mbed-dir', 'cmsis-nn', 'compiler_optimization',
       'FPU_status', 'model_type', 'weights', 'activations', 'pruned', 'layer',
       'layer_latency_mean', 'layer_latency_std', 'layer_latency_relativ',
       'error'],
      dtype='object')

Remove ResNet and L4 data

In [6]:
df = df.drop((df[df['error'].notna()]).index)

In [7]:
di = {"NUCLEO-L496ZG": "L4", "DISCO-F469NI": "F4", "NUCLEO-F767ZI": "F7"}

df = df.replace({"MCU": di})

In [8]:
_filter = ((df['model'] == '01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none') & (df['MCU'] == 'NUCLEO-L496ZG'))
display(df.loc[_filter])

,time,MCU,model,mbed-dir,cmsis-nn,compiler_optimization,FPU_status,model_type,weights,activations,pruned,layer,layer_latency_mean,layer_latency_std,layer_latency_relativ,error


## absolute

In [9]:
hue = 'cmsis-nn'
models = df['model'].unique()
mcus = df['MCU'].unique()
print(models)

filter_dict = {
    'compiler_optimization': '-Ofast',
    'FPU_status': 1
}


for model in models:
    for mcu in mcus:
        
        # skip ResNet float32 for L4 - didn't fit
        if ('01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none' in model) and ('L4' in mcu):
            continue
        
        model_name = get_model_name(model)

        filter = (df['model'] == model) & (df['MCU'] == mcu)
        filter_descr = ''
        for key in filter_dict:
            filter = filter & (df[key] == filter_dict[key])
            filter_descr += (str(filter_dict[key]) + '_')





        df_filtered = df.loc[filter]

        # let's sort the filtered df so that the order in the plots match
        model_type = df_filtered['model_type'].unique()[0]

        df_filtered = df_filtered.sort_values(by=['model_type', 'MCU'])

        df_filtered.loc[:,('layer_latency_mean')] /= 1000



        fig = sns.catplot(y='layer', x='layer_latency_mean', kind='bar', hue=hue, data=df_filtered)
        display(df_filtered[['layer', 'layer_latency_mean']])



        #plt.title(mcu + '-' + model_name + '-' + model_type)

        plt.xlabel("Latency of the Layer [ms]")
        plt.ylabel("Layer")


        filename = f'figures/layers/latency/{mcu}_{model_name}_{model_type}_absolute_hue-{hue}'

        print(filename)
        
        plt.legend(loc='upper right')

        plt.tight_layout()
        plt.show()
        plt.savefig(filename + '.pdf')
        tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

['LeNet-MNIST_none_tflite-builtins_none_none'
 'LeNet-MNIST_none_tflite-builtins-INT8_none_dataset'
 '01d_ResNet20_CIFAR-10_none_tflite-builtins-INT8_none_dataset'
 '01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,layer,layer_latency_mean
0,0_CONV_2D,41.2625
1,1_AVERAGE_POOL_2D,1.9139
2,2_CONV_2D,55.7670
3,3_AVERAGE_POOL_2D,0.7966
4,4_RESHAPE,0.3215
5,5_FULLY_CONNECTED,10.4366
6,6_FULLY_CONNECTED,1.5638
7,7_FULLY_CONNECTED,0.1438
8,8_SOFTMAX,0.0320
9,0_CONV_2D,41.2627


figures/layers/latency/L4_LeNet-MNIST_W-float32-A-float32_absolute_hue-cmsis-nn


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

,layer,layer_latency_mean
74,2_CONV_2D,1400.8518
75,3_ADD,17.2053
76,4_CONV_2D,1417.6337
77,5_CONV_2D,1261.9673
78,6_ADD,17.1351
...,...,...
135,29_ADD,0.4380
136,30_AVERAGE_POOL_2D,0.3330
137,31_RESHAPE,0.0220
138,32_FULLY_CONNECTED,0.0580


figures/layers/latency/F4_ResNet20_CIFAR-10_W-float32-A-float32_absolute_hue-cmsis-nn


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,layer,layer_latency_mean
74,2_CONV_2D,94.5944
75,3_ADD,0.9059
76,4_CONV_2D,94.4291
77,5_CONV_2D,94.4097
78,6_ADD,0.9217
...,...,...
135,29_ADD,0.2130
136,30_AVERAGE_POOL_2D,0.1262
137,31_RESHAPE,0.0091
138,32_FULLY_CONNECTED,0.0337


figures/layers/latency/F7_ResNet20_CIFAR-10_W-float32-A-float32_absolute_hue-cmsis-nn


## absolute (ResNet filtered)

In [10]:
hue = 'cmsis-nn'
models = df['model'].unique()
mcus = df['MCU'].unique()
print(models)

filter_dict = {
    'compiler_optimization': '-Ofast',
    'FPU_status': 1
}
layers = [
'0_QUANTIZE',
'1_CONV_2D',
'2_CONV_2D',
'3_CONV_2D',
'4_ADD',
'14_ADD',
'15_CONV_2D',
'16_CONV_2D',
'27_ADD',
'28_CONV_2D',
'29_CONV_2D',
'30_ADD',
'31_AVERAGE_POOL_2D',
'32_RESHAPE',
'33_FULLY_CONNECTED ',
'34_SOFTMAX',
'35_DEQUANTIZE',
]

for model in models:
    for mcu in mcus:
        
        # skip ResNet float32 for L4 - didn't fit
        if ('01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none' in model) and ('L4' in mcu):
            continue
        
        model_name = get_model_name(model)

        filter = (df['model'] == model) & (df['MCU'] == mcu)
        filter_descr = ''
        for key in filter_dict:
            filter = filter & (df[key] == filter_dict[key])
            filter_descr += (str(filter_dict[key]) + '_')


        if ('01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none' in model) or ('01d_ResNet20_CIFAR-10_none_tflite-builtins-INT8_none_dataset' in model):
            filter = filter * (df['layer'].isin(layers))



        df_filtered = df.loc[filter]

        # let's sort the filtered df so that the order in the plots match
        model_type = df_filtered['model_type'].unique()[0]

        df_filtered = df_filtered.sort_values(by=['model_type', 'MCU'])

        df_filtered.loc[:,('layer_latency_mean')] /= 1000



        fig = sns.catplot(y='layer', x='layer_latency_mean', kind='bar', hue=hue, data=df_filtered)
        display(df_filtered[['layer', 'layer_latency_mean']])



        #plt.title(mcu + '-' + model_name + '-' + model_type)

        plt.xlabel("Latency of the Layer [ms]")
        if "ResNet" in model:
            plt.ylabel("Selected Layers")
        else:
            plt.ylabel("Layers")


        filename = f'figures/layers/latency/{mcu}_{model_name}_{model_type}_absolute_hue-{hue}'

        print(filename)
        
        
        plt.legend(loc='lower right')

        plt.tight_layout()
        plt.show()
        plt.savefig(filename + '.pdf')
        tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

['LeNet-MNIST_none_tflite-builtins_none_none'
 'LeNet-MNIST_none_tflite-builtins-INT8_none_dataset'
 '01d_ResNet20_CIFAR-10_none_tflite-builtins-INT8_none_dataset'
 '01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,layer,layer_latency_mean
0,0_CONV_2D,41.2625
1,1_AVERAGE_POOL_2D,1.9139
2,2_CONV_2D,55.7670
3,3_AVERAGE_POOL_2D,0.7966
4,4_RESHAPE,0.3215
5,5_FULLY_CONNECTED,10.4366
6,6_FULLY_CONNECTED,1.5638
7,7_FULLY_CONNECTED,0.1438
8,8_SOFTMAX,0.0320
9,0_CONV_2D,41.2627


figures/layers/latency/L4_LeNet-MNIST_W-float32-A-float32_absolute_hue-cmsis-nn


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

figures/layers/latency/F4_ResNet20_CIFAR-10_W-float32-A-float32_absolute_hue-cmsis-nn


/usr/local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:177: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,layer,layer_latency_mean
74,2_CONV_2D,94.5944
87,15_CONV_2D,94.9486
100,28_CONV_2D,84.7078
107,1_CONV_2D,94.2989
108,2_CONV_2D,94.5909
121,15_CONV_2D,94.9153
134,28_CONV_2D,84.7086


figures/layers/latency/F7_ResNet20_CIFAR-10_W-float32-A-float32_absolute_hue-cmsis-nn


## relative

In [11]:
# model_type = 'W-int8-A-int8'
# model_type = 'W-float32-A-float32'

hue = 'cmsis-nn'
models = df['model'].unique()
print(models)

filter_dict = {
    #'MCU': 'NUCLEO-L496ZG',
    'compiler_optimization': '-Ofast',
    'FPU_status': 1
}


for model in models:

    model_name = get_model_name(model)
    
    
    
    filter = (df['model'] == model)
    filter_descr = ''
    for key in filter_dict:
        filter = filter & (df[key] == filter_dict[key])
        filter_descr += (str(filter_dict[key]) + '_')

        
    


    df_filtered = df.loc[filter]

    # let's sort the filtered df so that the order in the plots match
    model_type = df_filtered['model_type'].unique()[0]

    df_filtered = df_filtered.sort_values(by=['model_type', 'MCU'])




    fig = sns.catplot(y='layer', x='layer_latency_relativ', kind='bar', hue=hue, data=df_filtered)
    display(df_filtered[['layer', 'layer_latency_relativ']].drop_duplicates())



    #plt.title( model_name + '-' + model_type)

    plt.xlabel("Relative Latency of the Layer")
    plt.ylabel("Layers")
    

        
    plt.legend(loc='lower right')
    
    

    filename = f'figures/layers/latency/{model_name}_{model_type}_relative_hue-{hue}'

    print(filename)

    plt.tight_layout()
    plt.show()
    plt.savefig(filename + '.pdf')
    tikzplotlib.save(filename + '.tex', axis_width='280pt', axis_height=AXIS_HEIGHT)

['LeNet-MNIST_none_tflite-builtins_none_none'
 'LeNet-MNIST_none_tflite-builtins-INT8_none_dataset'
 '01d_ResNet20_CIFAR-10_none_tflite-builtins-INT8_none_dataset'
 '01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,layer,layer_latency_relativ
0,0_CONV_2D,0.23075704
1,1_AVERAGE_POOL_2D,0.02536069
2,2_CONV_2D,0.50457596
3,3_AVERAGE_POOL_2D,0.01078503
4,4_RESHAPE,0.00095540
5,5_FULLY_CONNECTED,0.19691741
6,6_FULLY_CONNECTED,0.02781954
7,7_FULLY_CONNECTED,0.00231449
8,8_SOFTMAX,0.00051445
9,0_CONV_2D,0.37543126


figures/layers/latency/LeNet-MNIST_W-float32-A-float32_relative_hue-cmsis-nn


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

,layer,layer_latency_relativ
0,0_QUANTIZE,0.00025828
1,1_CONV_2D,0.01793004
2,2_CONV_2D,0.06139891
3,3_CONV_2D,0.06143224
4,4_ADD,0.00213928
...,...,...
67,31_AVERAGE_POOL_2D,0.00020315
68,32_RESHAPE,0.00000603
69,33_FULLY_CONNECTED,0.00003918
70,34_SOFTMAX,0.00011637


figures/layers/latency/ResNet20_CIFAR-10_W-int8-A-int8_relative_hue-cmsis-nn


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,layer,layer_latency_relativ
74,2_CONV_2D,0.06656647
75,3_ADD,0.00081757
76,4_CONV_2D,0.06736392
77,5_CONV_2D,0.05996688
78,6_ADD,0.00081424
...,...,...
135,29_ADD,0.00013198
136,30_AVERAGE_POOL_2D,0.00007820
137,31_RESHAPE,0.00000564
138,32_FULLY_CONNECTED,0.00002088


figures/layers/latency/ResNet20_CIFAR-10_W-float32-A-float32_relative_hue-cmsis-nn


## relative (ResNet filtered)

In [12]:
# model_type = 'W-int8-A-int8'
# model_type = 'W-float32-A-float32'

hue = 'cmsis-nn'
models = df['model'].unique()
print(models)

layers = [
'0_QUANTIZE',
'1_CONV_2D',
'2_CONV_2D',
'3_CONV_2D',
'4_ADD',
'14_ADD',
'15_CONV_2D',
'16_CONV_2D',
'27_ADD',
'28_CONV_2D',
'29_CONV_2D',
'30_ADD',
'31_AVERAGE_POOL_2D',
'32_RESHAPE',
'33_FULLY_CONNECTED ',
'34_SOFTMAX',
'35_DEQUANTIZE',
]

filter_dict = {
    #'MCU': 'NUCLEO-L496ZG',
    'compiler_optimization': '-Ofast',
    'FPU_status': 1
}


for model in models:

    model_name = get_model_name(model)
    

            
    filter = (df['model'] == model)
    filter_descr = ''
    for key in filter_dict:
        filter = filter & (df[key] == filter_dict[key])
        filter_descr += (str(filter_dict[key]) + '_')

    if ('01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none' in model) or ('01d_ResNet20_CIFAR-10_none_tflite-builtins-INT8_none_dataset' in model):
        filter = filter * (df['layer'].isin(layers))
    


    df_filtered = df.loc[filter]

    # let's sort the filtered df so that the order in the plots match
    model_type = df_filtered['model_type'].unique()[0]

    df_filtered = df_filtered.sort_values(by=['model_type', 'MCU'])

    


    fig = sns.catplot(y='layer', x='layer_latency_relativ', kind='bar', hue=hue, data=df_filtered)
    display(df_filtered[['layer', 'layer_latency_relativ']].drop_duplicates())



    #plt.title( model_name + '-' + model_type)

 
    plt.xlabel("Relative Latency of the Layer")
    if "ResNet" in model:
        plt.ylabel("Selected Layers")
    else:
        plt.ylabel("Layers")

    plt.legend(loc='lower right')
    
    
    filename = f'figures/layers/latency/{model_name}_{model_type}_filtered_relative_hue-{hue}'

    print(filename)

    plt.tight_layout()
    plt.show()
    plt.savefig(filename + '.pdf')
    tikzplotlib.save(filename + '.tex', axis_width='280pt', axis_height=AXIS_HEIGHT)

['LeNet-MNIST_none_tflite-builtins_none_none'
 'LeNet-MNIST_none_tflite-builtins-INT8_none_dataset'
 '01d_ResNet20_CIFAR-10_none_tflite-builtins-INT8_none_dataset'
 '01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,layer,layer_latency_relativ
0,0_CONV_2D,0.23075704
1,1_AVERAGE_POOL_2D,0.02536069
2,2_CONV_2D,0.50457596
3,3_AVERAGE_POOL_2D,0.01078503
4,4_RESHAPE,0.00095540
5,5_FULLY_CONNECTED,0.19691741
6,6_FULLY_CONNECTED,0.02781954
7,7_FULLY_CONNECTED,0.00231449
8,8_SOFTMAX,0.00051445
9,0_CONV_2D,0.37543126


figures/layers/latency/LeNet-MNIST_W-float32-A-float32_filtered_relative_hue-cmsis-nn


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

figures/layers/latency/ResNet20_CIFAR-10_W-int8-A-int8_filtered_relative_hue-cmsis-nn


/usr/local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:177: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,layer,layer_latency_relativ
74,2_CONV_2D,0.06656647
87,15_CONV_2D,0.05113923
100,28_CONV_2D,0.04382677
107,1_CONV_2D,0.06184013
108,2_CONV_2D,0.06186207
121,15_CONV_2D,0.05657338
134,28_CONV_2D,0.05175413
74,2_CONV_2D,0.05861985
87,15_CONV_2D,0.05883934
100,28_CONV_2D,0.05249315


figures/layers/latency/ResNet20_CIFAR-10_W-float32-A-float32_filtered_relative_hue-cmsis-nn


### MCU as hue - relative

In [13]:
hue = 'MCU'
models = df['model'].unique()
print(models)

filter_dict = {
    'cmsis-nn': 'cmsis-nn',
    'compiler_optimization': '-Ofast',
    'FPU_status': 1
}


for model in models:

    model_name = get_model_name(model)
    
    filter = (df['model'] == model)
    filter_descr = ''
    for key in filter_dict:
        filter = filter & (df[key] == filter_dict[key])
        filter_descr += (str(filter_dict[key]) + '_')

        
    


    df_filtered = df.loc[filter]

    # let's sort the filtered df so that the order in the plots match
    model_type = df_filtered['model_type'].unique()[0]

    df_filtered = df_filtered.sort_values(by=['model_type', 'MCU'])




    fig = sns.catplot(y='layer', x='layer_latency_relativ', kind='bar', hue=hue, data=df_filtered)
    display(df_filtered[['layer', 'layer_latency_relativ']].drop_duplicates())



    plt.title( model_name + '-' + model_type)

    plt.xlabel("Relative Latency of the Layer")
    plt.ylabel("Layer")


    filename = f'figures/layers/latency/{model_name}_{model_type}_relative_hue-{hue}'

    print(filename)

    plt.tight_layout()
    plt.show()
    plt.savefig(filename + '.pdf')
    tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

['LeNet-MNIST_none_tflite-builtins_none_none'
 'LeNet-MNIST_none_tflite-builtins-INT8_none_dataset'
 '01d_ResNet20_CIFAR-10_none_tflite-builtins-INT8_none_dataset'
 '01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,layer,layer_latency_relativ
9,0_CONV_2D,0.37543126
10,1_AVERAGE_POOL_2D,0.01608892
11,2_CONV_2D,0.48997452
12,3_AVERAGE_POOL_2D,0.00666274
13,4_RESHAPE,0.00306859
14,5_FULLY_CONNECTED,0.09332929
15,6_FULLY_CONNECTED,0.01395277
16,7_FULLY_CONNECTED,0.00125456
17,8_SOFTMAX,0.00023735
9,0_CONV_2D,0.37233936


figures/layers/latency/LeNet-MNIST_W-float32-A-float32_relative_hue-MCU


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

,layer,layer_latency_relativ
36,0_QUANTIZE,0.00082574
37,1_CONV_2D,0.02215982
38,2_CONV_2D,0.05872511
39,3_CONV_2D,0.05881229
40,4_ADD,0.01626097
...,...,...
67,31_AVERAGE_POOL_2D,0.00020315
68,32_RESHAPE,0.00000603
69,33_FULLY_CONNECTED,0.00003918
70,34_SOFTMAX,0.00011637


figures/layers/latency/ResNet20_CIFAR-10_W-int8-A-int8_relative_hue-MCU


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,layer,layer_latency_relativ
106,0_CONV_2D,0.01765995
107,1_CONV_2D,0.06184013
108,2_CONV_2D,0.06186207
109,3_ADD,0.00029308
110,4_CONV_2D,0.06186207
...,...,...
135,29_ADD,0.00013198
136,30_AVERAGE_POOL_2D,0.00007820
137,31_RESHAPE,0.00000564
138,32_FULLY_CONNECTED,0.00002088


figures/layers/latency/ResNet20_CIFAR-10_W-float32-A-float32_relative_hue-MCU


### MCU as hue - relative (selected)

In [14]:
hue = 'MCU'
models = df['model'].unique()
print(models)

layers = [
'0_QUANTIZE',
'1_CONV_2D',
'2_CONV_2D',
'3_CONV_2D',
'4_ADD',
'14_ADD',
'15_CONV_2D',
'16_CONV_2D',
'27_ADD',
'28_CONV_2D',
'29_CONV_2D',
'30_ADD',
'31_AVERAGE_POOL_2D',
'32_RESHAPE',
'33_FULLY_CONNECTED ',
'34_SOFTMAX',
'35_DEQUANTIZE',
]

filter_dict = {
    'cmsis-nn': 'cmsis-nn',
    'compiler_optimization': '-Ofast',
    'FPU_status': 1
}


for model in models:

    model_name = get_model_name(model)
    
    filter = (df['model'] == model)
    filter_descr = ''
    for key in filter_dict:
        filter = filter & (df[key] == filter_dict[key])
        filter_descr += (str(filter_dict[key]) + '_')

    if ('01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none' in model) or ('01d_ResNet20_CIFAR-10_none_tflite-builtins-INT8_none_dataset' in model):
        filter = filter & (df['layer'].isin(layers)) 



    df_filtered = df.loc[filter]

    # let's sort the filtered df so that the order in the plots match
    model_type = df_filtered['model_type'].unique()[0]

    df_filtered = df_filtered.sort_values(by=['model_type', 'MCU'])




    fig = sns.catplot(y='layer', x='layer_latency_relativ', kind='bar', hue=hue, data=df_filtered, hue_order=["L4", "F4", "F7"])
    display(df_filtered[['layer', 'layer_latency_relativ']].drop_duplicates())



    #plt.title( model_name + '-' + model_type)

    plt.xlabel("Relative Latency of the Layer")
    if "ResNet" in model:
        plt.ylabel("Selected Layers")
    else:
        plt.ylabel("Layers")
    plt.legend(loc='lower right')


    filename = f'figures/layers/latency/{model_name}_{model_type}_filtered_relative_hue-{hue}'

    print(filename)

    plt.tight_layout()
    plt.show()
    plt.savefig(filename + '.pdf')
    tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

['LeNet-MNIST_none_tflite-builtins_none_none'
 'LeNet-MNIST_none_tflite-builtins-INT8_none_dataset'
 '01d_ResNet20_CIFAR-10_none_tflite-builtins-INT8_none_dataset'
 '01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,layer,layer_latency_relativ
9,0_CONV_2D,0.37543126
10,1_AVERAGE_POOL_2D,0.01608892
11,2_CONV_2D,0.48997452
12,3_AVERAGE_POOL_2D,0.00666274
13,4_RESHAPE,0.00306859
14,5_FULLY_CONNECTED,0.09332929
15,6_FULLY_CONNECTED,0.01395277
16,7_FULLY_CONNECTED,0.00125456
17,8_SOFTMAX,0.00023735
9,0_CONV_2D,0.37233936


figures/layers/latency/LeNet-MNIST_W-float32-A-float32_filtered_relative_hue-MCU


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

,layer,layer_latency_relativ
36,0_QUANTIZE,0.00082574
37,1_CONV_2D,0.02215982
38,2_CONV_2D,0.05872511
39,3_CONV_2D,0.05881229
40,4_ADD,0.01626097
50,14_ADD,0.00834843
51,15_CONV_2D,0.04937324
52,16_CONV_2D,0.04934398
63,27_ADD,0.00420114
64,28_CONV_2D,0.04559772


figures/layers/latency/ResNet20_CIFAR-10_W-int8-A-int8_filtered_relative_hue-MCU


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,layer,layer_latency_relativ
107,1_CONV_2D,0.06184013
108,2_CONV_2D,0.06186207
121,15_CONV_2D,0.05657338
134,28_CONV_2D,0.05175413
107,1_CONV_2D,0.05843161
108,2_CONV_2D,0.05861254
121,15_CONV_2D,0.05881355
134,28_CONV_2D,0.05248905


figures/layers/latency/ResNet20_CIFAR-10_W-float32-A-float32_filtered_relative_hue-MCU
